# Modelowanie liczby roszczeń

### Generowanie niejednorodnego procesu Poissona metodą przerzedzania

In [1]:
import numpy as np
import pandas as pd

In [86]:
def nhpp(lambda_func,lambda_func_params, t_max):
    """
    Funkcja generuje niejednorodny proces Poissona metodą przerzerzedzania.

    Argumenty:
        lambda_func: funkcja
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności
        t_max: float
            Maksymalny czas symulacji
        lambda_max: float
            Ograniczenie górne funkcji lambda_func
    Zwraca:
        numpy.ndarray
            Tablica czasów.
    """

    S = 0
    N = 0
    S_tab = [0]
    lambda_max = lambda_func(0,lambda_func_params)[1]
    
    while S < t_max:
        U = np.random.uniform(0,1)
        S=S+(-1/lambda_max)*np.log(U)
        if S > t_max:
            break
        U = np.random.uniform(0,1)
        if U <= lambda_func(S,lambda_func_params)[0]/lambda_max:
            N = N + 1
            S_tab.append(S)

    return N, S_tab

In [97]:
def sin_int_fun(t,lambda_func_params):
    """
    Sinusoidalna funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności.
            W tym przypadku listę zawierającą: [lambda_0, lambda_1, T, phi]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0+lambda_1.    
    """
    return lambda_func_params[0]+lambda_func_params[1]*np.sin(2*np.pi*t/lambda_func_params[2]+lambda_func_params[3]), lambda_func_params[0]+lambda_func_params[1]

In [82]:
def exp_int_fun(t,lambda_func_params):
    """
    Wykładnicza funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności. 
            W tym przypadku listę zawierającą: [lambda_0, lambda]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0/lambda.
    """
    return lambda_func_params[0]*np.exp(-lambda_func_params[1]*t), lambda_func_params[0]*lambda_func_params[1]

In [87]:
nhpp(sin_int_fun,[1,3,1,0],10)

(14,
 [0,
  2.2883348786553515,
  2.345736718989954,
  4.134051936061515,
  4.176628344998191,
  4.236621035082244,
  6.140905302459069,
  6.157262016676862,
  6.3371516837784245,
  7.143012732508846,
  7.215260841709861,
  8.004968592017143,
  9.240759544898902,
  9.381962467442568,
  9.525352682936294])

In [88]:
nhpp(exp_int_fun,[2,1/10],100)

(4,
 [0,
  6.018626937599767,
  10.064875370089876,
  18.28310918285859,
  26.536471274329607])

Empiryczny proces $N_t$.

In [6]:
data = pd.read_csv("https://github.com/ndzadz/mgr/blob/main/data_us_1990_2022.csv?raw=true",encoding="latin1",sep=";",header=0)
p95 = np.percentile(data[data["Insured Damages, Adjusted (\'000 US$)"].isna()==0]["Insured Damages, Adjusted (\'000 US$)"],95)
data = data[(data["Insured Damages, Adjusted (\'000 US$)"].isna()==0)&(data["Insured Damages, Adjusted (\'000 US$)"]<p95)]
data = data.rename(columns={'ï»¿Dis No':'Dis No'})

In [51]:
N = pd.concat([pd.Series([0]),pd.Series(data.groupby('Year').count()['Dis No'])]).reset_index()[0].rename('N')

In [52]:
N

0      0
1     17
2     28
3     29
4     25
5     11
6     18
7     12
8     26
9     26
10    16
11     7
12    10
13    13
14    13
15    13
16     9
17    12
18    22
19    13
20    10
21     9
22    16
23    20
24    20
25    15
26    25
27    21
28    32
29     4
30    10
31     8
32     1
33     2
Name: N, dtype: int64

Metoda najmniejszej sumy kwadratów

In [77]:
from scipy.optimize import minimize

Sinusoidalna funkcja intensywności

In [119]:
def sum_of_squares_sin(lambda_func_params):
    t = np.array(range(len(N)))
    return np.sum(np.array((N-sin_int_fun(t, lambda_func_params)[0])**2))

In [120]:
sum_of_squares_sin([1,3,1,0])

9015.000000000011

In [124]:
bnds = ((0, None), (0, None), (0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0, 1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_sin, params, bounds=bnds)
print(result)


      fun: 1575.267861512148
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.27373657e-05, -2.27373677e-05,  7.44648792e-02, -1.81898925e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 290
      nit: 44
     njev: 58
   status: 0
  success: True
        x: array([14.51045062,  6.56946102,  1.0560816 ,  9.69327362])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [129]:
n = nhpp(sin_int_fun,result.x,33)
print(n)

(472, [0, 0.08981736392816568, 0.14397341816163373, 0.18763563732050975, 0.2537486430831448, 0.2785173508023243, 0.38337861161231196, 0.43877051411014645, 0.5598194933578476, 0.6268630686613095, 0.7049046689374683, 0.7228737597426061, 0.7468563702586327, 0.7808411173944038, 0.7874667603128714, 0.8440398361636295, 0.8808722936561352, 1.1444115557226653, 1.1730221075133942, 1.4227634223496195, 1.4608808374963587, 1.6068613110762362, 1.7035449637376445, 1.7155317558770973, 1.8556786176078695, 1.8566772427642009, 1.9463833177684262, 2.0617657522265573, 2.2745459859335577, 2.41294015771663, 2.600727261010273, 2.6533948572011012, 2.7953729334265716, 2.8110705104578058, 2.877613569457601, 2.880630493480628, 2.906410238591132, 2.911434962084891, 2.9117943026217246, 3.092902340692061, 3.099714125712371, 3.148367079277057, 3.2093114340073363, 3.2216196597714273, 3.2736434873898483, 3.3207828076457733, 3.341815232926383, 3.3720830168384315, 3.4339746097651314, 3.468235183872784, 3.594799410730414

In [144]:
unique, counts = np.unique(np.ceil(n[1]), return_counts=True)
N_t = pd.Series(counts)
N_t[0]=0
N_t

0      0
1     16
2     10
3     12
4     22
5     18
6     15
7     10
8     12
9     13
10    17
11    15
12     8
13    23
14    20
15    15
16    17
17     6
18    16
19    17
20    17
21    15
22    18
23    14
24    13
25    11
26    10
27    15
28    15
29    15
30    11
31    12
32     8
33    16
dtype: int64

Wykładnicza funkcja intensywności

In [146]:
def sum_of_squares_exp(lambda_func_params):
    t = np.array(range(len(N)))
    return np.sum(np.array((N-exp_int_fun(t, lambda_func_params)[0])**2))

In [147]:
sum_of_squares_exp([1,3])

10006.16344218761

In [149]:
bnds = ((0, None), (0, None)) # ograniczenia na wartości parametrów
params = [1.0, 1.0] # Przyjmujemy początkowe wartości parametrów
result = minimize(sum_of_squares_exp, params, bounds=bnds)
print(result)


      fun: 2130.1683437993447
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0001819,  0.0721684])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 51
      nit: 9
     njev: 17
   status: 0
  success: True
        x: array([1.85912816e+01, 1.31208572e-02])


Symulacja procesu z sinusoidalną funkcją intensywności dla wyliczonych parametrów

In [150]:
n = nhpp(exp_int_fun,result.x,33)
print(n)

(4, [0, 2.210059237127692, 3.0576332719452273, 29.990610443917966, 31.1412473300267])


In [151]:
unique, counts = np.unique(np.ceil(n[1]), return_counts=True)
N_t = pd.Series(counts)
N_t[0]=0
N_t

0    0
1    1
2    1
3    1
4    1
dtype: int64